# Heart Disease - Lab 1
Martin Garcia, Joanna Duran, Daniel Byrne

### Business Understanding

### Data Meaning Type

### Data Quality

### Simple Statistics

### Visualize Attributes

### Explore Joint Attributes and Class

### New Features

### Exceptional Work